# Join and preprocess multiple time series data

## Load data

In [18]:
print(1+1)


2


In [19]:
import yfinance as yf
df = yf.download(tickers='MSFT', start='2000-01-01', end=None)

df

/tmp/ipykernel_1265/3428693966.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Date,,,,,
2000-01-03,35.601460,36.231405,34.207944,35.849620,53228400
2000-01-04,34.398834,35.773260,34.284299,34.685173,54119000
...,...,...,...,...,...
2025-12-05,483.160004,483.399994,478.880005,482.519989,22587400
2025-12-08,491.019989,492.299988,484.380005,484.885010,20031253


In [20]:
df['Close'].plot()

In [21]:
import pandas as pd

path = "../../data/FRED/CORESTICKM159SFRBATL.csv"
df = pd.read_csv(path, index_col='observation_date')

df

,CORESTICKM159SFRBATL
observation_date,
1968-01-01,3.651861
1968-02-01,3.673819
...,...
2025-08-01,3.400000
2025-09-01,3.326080


In [22]:
df.plot()

## Parse dates to join time series

In [23]:
path = "../../data/FRED/CORESTICKM159SFRBATL.csv"
df1 = pd.read_csv(path, index_col='observation_date', parse_dates=True)

df1

,CORESTICKM159SFRBATL
observation_date,
1968-01-01,3.651861
1968-02-01,3.673819
...,...
2025-08-01,3.400000
2025-09-01,3.326080


In [24]:
mortgagepath = '../../data/FRED/MORTGAGE30US.csv'
df2 = pd.read_csv(mortgagepath, index_col='observation_date', parse_dates=True)
df2

,MORTGAGE30US
observation_date,
1971-04-02,7.33
1971-04-09,7.31
...,...
2025-11-26,6.23
2025-12-04,6.19


In [31]:

df3 = pd.concat(objs=[df1, df2], axis=1)
df3.head(5)

,CORESTICKM159SFRBATL,MORTGAGE30US
observation_date,,
1968-01-01,3.651861,NaN
1968-02-01,3.673819,NaN
1968-03-01,4.142164,NaN
1968-04-01,4.155828,NaN
1968-05-01,4.088245,NaN


In [26]:
df3.plot()

## Inner join vs outer join

In [28]:
df3 = pd.concat(objs=[df1, df2], axis=1, join='inner')
df3.tail(7)

,CORESTICKM159SFRBATL,MORTGAGE30US
observation_date,,
2021-07-01,2.330438,2.98
2022-09-01,6.344649,5.66
2022-12-01,6.545514,6.49
2023-06-01,5.588761,6.79
2024-02-01,4.400073,6.63
2024-08-01,4.112096,6.73
2025-05-01,3.159131,6.76


In [29]:
df3.plot()

## Interpolate missing values

In [33]:
df3 = df3.interpolate(method='linear')

,CORESTICKM159SFRBATL,MORTGAGE30US
observation_date,,
1968-01-01,3.651861,NaN
1968-02-01,3.673819,NaN
...,...,...
2025-09-04,3.419811,6.50
2025-09-11,3.419811,6.35


In [34]:
df3.plot()

## Iterate for seamless load

In [52]:
paths = ['../../data/FRED/CORESTICKM159SFRBATL.csv',
         '../../data/FRED/FEDFUNDS.csv',
         '../../data/FRED/MORTGAGE30US.csv',
         '../../data/FRED/T10YIE.csv',
         '../../data/FRED/UNRATE.csv']

dfs = []
for f in paths:
    df = pd.read_csv(f,index_col=0,parse_dates=True)
    dfs.append(df)

final_df = pd.concat(dfs,axis=1)
final_df = final_df.interpolate(mode='linear')
final_df
    

,CORESTICKM159SFRBATL,FEDFUNDS,MORTGAGE30US,T10YIE,UNRATE
observation_date,,,,,
1948-01-01,NaN,NaN,NaN,NaN,3.4
1948-02-01,NaN,NaN,NaN,NaN,3.8
...,...,...,...,...,...
2025-12-04,3.32608,3.88,6.19,2.26,4.4
2025-12-05,3.32608,3.88,6.19,2.26,4.4


In [53]:


final_df.plot()

## Export data

,CORESTICKM159SFRBATL,MORTGAGE30US,FEDFUNDS,T10YIE,UNRATE
date,,,,,
1948-01-01,NaN,NaN,NaN,NaN,3.4
1948-02-01,NaN,NaN,NaN,NaN,3.8
...,...,...,...,...,...
2025-09-11,3.419811,6.35,4.33,2.34,4.3
2025-09-12,3.419811,6.35,4.33,2.36,4.3


,CPI,MR,FFR,INF10,UNRT
date,,,,,
1948-01-01,NaN,NaN,NaN,NaN,3.4
1948-02-01,NaN,NaN,NaN,NaN,3.8
...,...,...,...,...,...
2025-09-11,3.419811,6.35,4.33,2.34,4.3
2025-09-12,3.419811,6.35,4.33,2.36,4.3
